In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import copy

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']

# 開始框acupoint

In [4]:
jsons = "../2_json"

names = os.listdir("../0_original_video")

for name in names:
    jsons_name = os.path.join(jsons,name)
    video_name= "../0_original_video/{names}".format(names=name)
    
    for deg in degrees:
        jsons_name_deg = os.path.join(jsons_name, deg)
        if not os.path.isdir(jsons_name_deg):
            os.makedirs(jsons_name_deg)
    
        video_name_deg = os.path.join(video_name,deg)
        video = glob.glob(os.path.join(video_name_deg, "resize.mp4"))
#         count = 0
        mask_2_point = {}                 #裝每個frame追蹤的左上點和右下點
        visible = {}                      #看針灸點是否可見
        acupoint_location = {} 
        

        

        set_all_tracker_flag = False
        for j in range(0,7):

            tracker_list = []
            tracker_success = []
            for i in range(0, 3):
                tracker = cv2.TrackerCSRT_create()   
                tracker_list.append(tracker)
                tracker_success.append(1)
            set_all_tracker_flag = False  #第一個FRAME先暫停，並框ROI用的FLAG
            colors = [(255,0,0)]

            cap = cv2.VideoCapture(video[0])            # 讀取某個影片
            
            temp = 0
            
            if not cap.isOpened():
                print("Cannot open camera")
                exit()

            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Cannot receive frame")
                    break
                keyName = cv2.waitKey(1)

                frame_index = "frame_{f_index}".format(f_index = str(temp))

                if frame_index not in mask_2_point:  #存mask的地方
                    mask_2_point[frame_index] = {}

                if frame_index not in visible:       #存keypoint可不可以被看到
                    visible[frame_index] = {}

                if frame_index not in acupoint_location:    #存keypoint位置
                    acupoint_location[frame_index] = {}


                visual_frame = copy.deepcopy(frame)

                if keyName == ord('q'):
                    break

                if set_all_tracker_flag == False:
                    # 如果尚未開始追蹤，就開始標記追蹤物件的外框
                    for i in range(0,3):
                        print("框第{n}個穴道".format(n = 3*j + i))
                        area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=True)
                        tracker_list[i].init(frame, area)    # 初始化追蹤器
                    set_all_tracker_flag = True            # 設定可以開始追蹤
                if set_all_tracker_flag:
                    for i in range(0,3):
                        success, point = tracker_list[i].update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                        if success and tracker_success[i] == 1:
                            p1 = (int(point[0]), int(point[1]))
                            p2 = (int(point[0] + point[2]), int(point[1] + point[3]))


                            center_point = ( int((p1[0] + p2[0]) / 2) , int((p1[1] + p2[1]) / 2))


                            mask_2_point[frame_index][str(3*j + i)] = {}
                            mask_2_point[frame_index][str(3*j + i)]['left_top'] = p1
                            mask_2_point[frame_index][str(3*j + i)]['right_bot'] = p2

                            visible[frame_index][str(3*j + i)] = {}
                            visible[frame_index][str(3*j + i)]['visible'] = 2

                            acupoint_location[frame_index][str(3*j + i)] = {}
                            acupoint_location[frame_index][str(3*j + i)]['keypoint_location'] = center_point


                            cv2.circle(visual_frame, center_point, 1, (0, 0, 255), 4) 

                        else:
                            mask_2_point[frame_index][str(3*j + i)] = {}
                            mask_2_point[frame_index][str(3*j + i)]['left_top'] = (-1, -1)
                            mask_2_point[frame_index][str(3*j + i)]['right_bot'] = (-1, -1)

                            visible[frame_index][str(3*j + i)] = {}
                            visible[frame_index][str(3*j + i)]['visible'] = 1

                            acupoint_location[frame_index][str(3*j + i)] = {}
                            acupoint_location[frame_index][str(3*j + i)]['keypoint_location'] = (-1, -1)

                            tracker_success[i] = 0




                cv2.imshow('oxxostudio', visual_frame)
    #             cv2.imshow('123',frame)

                temp = temp + 1
            cap.release()
            cv2.destroyAllWindows()






            with open( jsons_name_deg + "/mask.json", "w") as mask_file:
                json.dump(mask_2_point, mask_file, indent = 4)


            with open( jsons_name_deg + "/visible.json", "w") as visible_file:
                json.dump(visible, visible_file, indent = 4)


            with open( jsons_name_deg + "/keypoint_location.json", "w") as acupoint_location_file:
                json.dump(acupoint_location, acupoint_location_file, indent = 4)

            cap.release()
            cv2.destroyAllWindows()




框第0個穴道
框第1個穴道
框第2個穴道
Cannot receive frame
框第3個穴道
框第4個穴道
框第5個穴道
Cannot receive frame
框第6個穴道
框第7個穴道
框第8個穴道
Cannot receive frame
框第9個穴道
框第10個穴道
框第11個穴道
Cannot receive frame
框第12個穴道
框第13個穴道
框第14個穴道
Cannot receive frame
框第15個穴道
框第16個穴道
框第17個穴道
Cannot receive frame
框第18個穴道
框第19個穴道
框第20個穴道
Cannot receive frame


NameError: name 'a' is not defined

In [4]:
name = "daniel"
video_name= r"D:\keypoint\original_video\{names}".format(names = name)
each_training_json_name = r"D:\keypoint\each_training_json\{names}".format(names = name)
crop_512_for_inpainting_name = r"D:\keypoint\crop_512_for_inpainting\{names}".format(names = name)
mask_name = r"D:\keypoint\mask\{names}".format(names = name)
crop_512_center_json_name = r"D:\keypoint\crop_512_center_json\{names}".format(names = name)


for deg in degrees:
    count = 0
    video_name_deg = os.path.join(video_name, deg)
    each_training_json_name_deg = os.path.join(each_training_json_name, deg)
    crop_512_for_inpainting_name_deg = os.path.join(crop_512_for_inpainting_name, deg)
    crop_512_center_json_name_deg = os.path.join(crop_512_center_json_name, deg)
    
    
    if os.path.exists(os.path.join(each_training_json_name_deg, "keypoint_location.json")):
        print(deg)
        
        with open(os.path.join(each_training_json_name_deg, "mask.json"), "r") as f_mask:
            mask = json.load(f_mask)
        with open(os.path.join(each_training_json_name_deg, "visible.json"), "r") as f_visible:
            visible = json.load(f_visible)
        with open(os.path.join(each_training_json_name_deg, "keypoint_location.json"), "r") as f_keypoint_location:
            keypoint_location = json.load(f_keypoint_location)
        with open(os.path.join(crop_512_center_json_name_deg, "512_size_center.json"), "r") as f_center_file:
            center_of_512 = json.load(f_center_file)
        
        
        
        
        
        video = os.path.join(video_name_deg, "resize.mp4")
        cap = cv2.VideoCapture(video)
        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Cannot receive frame")
                break
            cv2.imshow("123",frame)
            frame_512 = frame[center_of_512[1] - 256 : center_of_512[1] + 256, center_of_512[0] - 256 : center_of_512[0] + 256]
            mask_ori_size = 255 - np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
            for i in range(0,len(mask["frame_0"])):
                imgRoi = frame[mask["frame_"+str(count)][str(i)]['left_top'][1] : mask["frame_"+str(count)][str(i)]['right_bot'][1], mask["frame_"+str(count)][str(i)]['left_top'][0] : mask["frame_"+str(count)][str(i)]['right_bot'][0]]  # imgRoi = frame[p1y:p2y,p1x:p2x]
                imgRoi_mask = np.zeros((imgRoi.shape[0], imgRoi.shape[1]), np.uint8)

                mask_ori_size[mask["frame_"+str(count)][str(i)]['left_top'][1] : mask["frame_"+str(count)][str(i)]['right_bot'][1], mask["frame_"+str(count)][str(i)]['left_top'][0] : mask["frame_"+str(count)][str(i)]['right_bot'][0]] = imgRoi_mask
            mask_512 = mask_ori_size[center_of_512[1] - 256 : center_of_512[1] + 256, center_of_512[0] - 256 : center_of_512[0] + 256]

            cv2.imshow('oxxostudio', frame_512)
            cv2.imshow('123',mask_512)
            
 
            if not os.path.exists(crop_512_for_inpainting_name):
                os.mkdir(crop_512_for_inpainting_name)   
            crop_512_for_inpainting_name_deg = os.path.join(crop_512_for_inpainting_name, deg)
            if not os.path.exists(crop_512_for_inpainting_name_deg):
                os.mkdir(crop_512_for_inpainting_name_deg)
            
            
            if not os.path.exists(mask_name):
                os.mkdir(mask_name)
            mask_name_deg = os.path.join(mask_name, deg)
            if not os.path.exists(mask_name_deg):
                os.mkdir(mask_name_deg)  
            
            
            cv2.imwrite(os.path.join(crop_512_for_inpainting_name_deg, 'test{index}.png'.format(name_0 = name, index = str(count))), frame_512)
            cv2.imwrite(os.path.join(mask_name_deg, 'mask{index}.png'.format(name_0 = name, index = str(count))), mask_512)
            count = count + 1
            
            
        cap.release()
        cv2.destroyAllWindows()

15cm_0mm_0deg
Cannot receive frame
15cm_25mm_5deg
Cannot receive frame
20cm_0mm_0deg
Cannot receive frame
20cm_25mm_5deg
Cannot receive frame


In [3]:


name = "daniel"
video_name= r"D:\keypoint\original_video\{names}".format(names = name)
each_training_json_name = r"D:\keypoint\each_training_json\{names}".format(names = name)

degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']


for deg in degrees:
    count = 0
    bbox = {}
    video_name_deg = os.path.join(video_name, deg)
    each_training_json_name_deg = os.path.join(each_training_json_name, deg)
    
    
    if os.path.exists(os.path.join(each_training_json_name_deg, "keypoint_location.json")):
        print(deg)
        
        video = os.path.join(video_name_deg, "resize.mp4")
        cap = cv2.VideoCapture(video)
    
        tracker = cv2.TrackerCSRT_create()   

        set_all_tracker_flag = False  #第一個FRAME先暫停，並框ROI用的FLAG
        colors = [(255,0,0)]

        
        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Cannot receive frame")
                break
            keyName = cv2.waitKey(1)

            frame_index = "frame_{f_index}".format(f_index = str(count))

            visual_frame = copy.deepcopy(frame)

            if keyName == ord('q'):
                break

            if set_all_tracker_flag == False:
                # 如果尚未開始追蹤，就開始標記追蹤物件的外框
                area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=False)
                tracker.init(frame, area)    # 初始化追蹤器
                set_all_tracker_flag = True            # 設定可以開始追蹤
            if set_all_tracker_flag:
                success, point = tracker.update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                if success:
                    p1 = (int(point[0]), int(point[1]))
                    p2 = (int(point[0] + point[2]), int(point[1] + point[3]))





                    bbox[frame_index] = (int(point[0]), int(point[1]), int(point[2]), int(point[3]))

                    cv2.rectangle(visual_frame, p1, p2, (0, 0, 255), 4) 





                cv2.imshow('oxxostudio',visual_frame)

                count = count + 1
            with open(r"D:\keypoint\each_training_json\{names}\{degrees}\bbox.json".format(names = name, degrees = deg), "w") as bbox_file:
                json.dump(bbox, bbox_file, indent = 4)



        cap.release()
        cv2.destroyAllWindows()

15cm_0mm_0deg
Cannot receive frame
15cm_25mm_5deg
Cannot receive frame
20cm_0mm_0deg
Cannot receive frame
20cm_25mm_5deg
Cannot receive frame
